In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('UCF/train_new.csv')
train.head()

,image,class
0,CBD\v_bend_g01_c01.avi_frame0.jpg,bend
1,CBD\v_bend_g01_c01.avi_frame1.jpg,bend
2,CBD\v_bend_g01_c01.avi_frame2.jpg,bend
3,CBD\v_bend_g01_c01.avi_frame3.jpg,bend
4,CBD\v_bend_g02_c01.avi_frame0.jpg,bend


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████████████████████████████████████████████████████████████████████████| 3951/3951 [00:51<00:00, 77.20it/s]


(3951, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [7]:
#X_train = base_model.predict(X_train)
X_train = base_model.predict(X_train)
X_train.shape#

(3160, 7, 7, 512)

In [8]:
#X_test = base_model.predict(X_test)
X_test = base_model.predict(X_test)
X_test.shape

(791, 7, 7, 512)

In [10]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(3160, 7*7*512)
X_test = X_test.reshape(791, 7*7*512)

In [11]:
# normalizing the pixel values
print('Before normalization : Min={}, max={}'.format(X_train.min(),X_train.max()))
max = X_train.max()
X_train = X_train/max
X_test = X_test/max
print('After normalization  : Min={}, max={}'.format(X_train.min(),X_train.max()))

Before normalization : Min=0.0, max=11.176948547363281
After normalization  : Min=0.0, max=1.0


In [12]:
# shape of images
X_train.shape

(3160, 25088)

In [17]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu',input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [22]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [23]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [24]:
# training the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=300)

Epoch 1/20
11/11 [==============================] - 5s 480ms/step - loss: 0.0658 - accuracy: 0.9791 - val_loss: 0.0572 - val_accuracy: 0.9798
Epoch 2/20
11/11 [==============================] - 4s 375ms/step - loss: 0.0568 - accuracy: 0.9791 - val_loss: 0.0688 - val_accuracy: 0.9823
Epoch 3/20
11/11 [==============================] - 5s 415ms/step - loss: 0.0573 - accuracy: 0.9807 - val_loss: 0.0743 - val_accuracy: 0.9785
Epoch 4/20
11/11 [==============================] - 6s 528ms/step - loss: 0.0518 - accuracy: 0.9816 - val_loss: 0.0542 - val_accuracy: 0.9810
Epoch 5/20
11/11 [==============================] - 4s 391ms/step - loss: 0.0501 - accuracy: 0.9810 - val_loss: 0.0719 - val_accuracy: 0.9785
Epoch 6/20
11/11 [==============================] - 5s 460ms/step - loss: 0.0515 - accuracy: 0.9823 - val_loss: 0.0502 - val_accuracy: 0.9798
Epoch 7/20
11/11 [==============================] - 4s 394ms/step - loss: 0.0478 - accuracy: 0.9791 - val_loss: 0.0507 - val_accuracy: 0.9798
Epoch 